# Affordability
wrritten by: **Hanshi Tang**<br>

Income-to-Rent Ratio, IRR


In [74]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
from sklearn.preprocessing import MinMaxScaler

import geopandas as gpd
import folium
from folium import GeoJsonTooltip


In [75]:
# load data
df = pd.read_parquet("../data/curated/features_domain.parquet") 

In [76]:
# Set display option to show all columns
pd.set_option('display.max_columns', None)

In [77]:
# Display the first 5 rows of the dataframe
df.head()

,url,price,address,property_type,latitude,longitude,Beds,Baths,Parking,bond,extracted_price,geometry,index_right,sa2_code,sa2_name,chg_flag,chg_lbl,sa3_code,sa3_name,sa4_code,sa4_name,gcc_code,gcc_name,ste_code,ste_name,aus_code,aus_name,areasqkm,loci_uri,geometry_proj,suburb,property_id,nearest_station_id,train_station_distance_km,cbd_distance_km,nearest_hospital_id,nearest_hospital_distance,mean_stores,total_stores,count_shopping_centres,SA2 code_x,Region,ERP_2021,ERP_2026,ERP_2031,ERP_2036,PNPD_2021,PNPD_2026,PNPD_2031,PNPD_2036,POPD_2021,POPD_2026,POPD_2031,POPD_2036,OPD_2021,OPD_2026,OPD_2031,OPD_2036,HHS_2021,HHS_2026,HHS_2031,HHS_2036,SPD_2021,SPD_2026,SPD_2031,SPD_2036,OCC_2021,OCC_2026,OCC_2031,OCC_2036,population_growth_2021_2026,population_growth_2026_2031,population_growth_2031_2036,total_population_growth,Facility Count,erp_2022_no.,erp_2023_no.,erp_change_no.,erp_change_percentage,pop_change_natural_increase,pop_change_net_internal_migration,pop_change_oversea_migration,area_km2,population_density_persons_per_km2_2023,property_sa2_name,nearest_parkres_distance,nearest_parkres_name,parkres_sa2_name,closest_school,distance_to_closest_school,closest_independent_school,distance_to_closest_independent_school,total_value_of_items,total_number_of_items,crime_frequency,SA2 code_y,SA2 name,Earners,Median age of earners,Sum,Median,Mean,P80/P20,P80/P50,P20/P50,P10/P50,Gini coefficient,Top 1%,Top 5%,Top 10%,Lowest Quartile,Second Quartile,Third Quartile,Highest Quartile
0,https://www.domain.com.au/10-allara-court-donv...,"$1,400.00","10 Allara Court, Donvale VIC 3111",Townhouse,-37.774273,145.181126,4.0,3.0,2.0,9125.0,1400.0,POINT (145.1811258 -37.77427300000001),310.0,211021261,Donvale - Park Orchards,0,No change,21102,Manningham - East,211,Melbourne - Outer East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.8028,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (989259.4173701586 -4291898.086618988),Donvale,0,19900,5.6596,25.93487,H5063,1.920280,NaN,NaN,NaN,211021261,Donvale - Park Orchards,16410.0,16758.344317,17104.792858,17452.526326,268.029273,295.811538,329.732136,361.211596,16141.970727,16462.532778,16775.060722,17091.31473,5719.325815,5867.52,6059.52,6251.52,2.822356,2.805705,2.768381,2.733945,5913,6112.0,6312.0,6512.0,0.967246,0.96,0.96,0.96,2.122756,2.06732,2.032959,6.223035,NaN,16405,16511,106,0.6,-77,-59,242,20.8,793.7,Donvale - Park Orchards,3541.030,Mullum Mullum Park,Donvale - Park Orchards,1759,0.345483,1724,1.086111,NaN,NaN,NaN,211021261,Donvale - Park Orchards,10426,48,872950324,53881,83728,6.95,2.24,0.32,0.11,0.555,13.1,27.9,39.7,28.5,22.1,19.0,30.4
1,https://www.domain.com.au/7-pine-ridge-donvale...,$750 per week,"7 Pine Ridge, Donvale VIC 3111",House,-37.791251,145.175649,4.0,2.0,0.0,3259.0,750.0,POINT (145.1756489 -37.7912513),310.0,211021261,Donvale - Park Orchards,0,No change,21102,Manningham - East,211,Melbourne - Outer East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,20.8028,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (988607.8235432974 -4293742.529802319),Donvale,1,19899,4.6923,23.77997,H5063,0.384624,NaN,NaN,NaN,211021261,Donvale - Park Orchards,16410.0,16758.344317,17104.792858,17452.526326,268.029273,295.811538,329.732136,361.211596,16141.970727,16462.532778,16775.060722,17091.31473,5719.325815,5867.52,6059.52,6251.52,2.822356,2.805705,2.768381,2.733945,5913,6112.0,6312.0,6512.0,0.967246,0.96,0.96,0.96,2.122756,2.06732,2.032959,6.223035,NaN,16405,16511,106,0.6,-77,-59,242,20.8,793.7,Donvale - Park Orchards,2536.544,Mullum Mullum Park,Donvale - Park Orchards,5019,0.986444,1724,2.472361,NaN,NaN,NaN,211021261,Donvale - Park Orchards,10426,48,872950324,53881,83728,6.95,2.24,0.32,0.11,0.555,13.1,27.9,39.7,28.5,22.1,19.0,30.4
2,https://www.domain.com.au/20-mulsanne-way-donv...,$1300 per week,"20 Mulsanne Way, Donvale VIC 3111",House,-37.797232,145.181264,5.0,2.0,2.0,5649.0,1300.0,POINT (145.1812636 -37.7972323),310.0,211021261,Donvale - Park Orchards,0,No change,21102,Man

In [78]:
# check median income for whole dataset
df['Median'].describe()

count     9565.000000
mean     56147.471720
std      11560.078243
min      23341.000000
25%      49545.000000
50%      55990.000000
75%      64287.000000
max      81648.000000
Name: Median, dtype: float64

In [79]:
# Aggregate data by SA2 area
grouped_df = df.groupby(['sa2_code', 'sa2_name']).agg({
    'Median': 'median',
    'extracted_price': 'mean',
    'ERP_2021': 'sum',
    'OCC_2021': 'sum',
    'population_density_persons_per_km2_2023': 'mean',
    'Gini coefficient': 'mean'
}).reset_index()

In [80]:
# Define features
features = ['sa2_code', 'sa2_name', 'Median', 'extracted_price', 'ERP_2021', 'OCC_2021', 'population_density_persons_per_km2_2023', 'Gini coefficient']
grouped_df = grouped_df[features]

In [81]:
# Calculate Income-to-Price Ratio (IPR)
# IPR measures the affordability of housing relative to income. Higher values indicate better affordability.
grouped_df['IPR'] = grouped_df['Median'] / grouped_df['extracted_price']

# Calculate Housing Pressure Index (HPI)
# HPI measures the availability of housing. Higher values indicate less pressure on housing supply, which means better affordability.
grouped_df['HPI'] = 1 - (grouped_df['OCC_2021'] / grouped_df['ERP_2021'])

# Calculate Population Density Inverse (PDI)
# PDI is the inverse of population density. Lower population density can imply more affordable housing options.
grouped_df['PDI'] = 1 / grouped_df['population_density_persons_per_km2_2023']

# Calculate Gini Inequality Inverse (GII)
# GII represents income equality. Higher values indicate less income inequality, which is generally associated with better affordability.
grouped_df['GII'] = 1 - grouped_df['Gini coefficient']

In [82]:
# Select the newly created columns for affordability calculation
affordability_features = ['IPR', 'HPI', 'PDI', 'GII']

In [83]:
# remove inf values
grouped_df = grouped_df.replace([np.inf, -np.inf], np.nan)

In [84]:
# Standardize features using Min-Max scaling
scaler = MinMaxScaler()
grouped_df[affordability_features] = scaler.fit_transform(grouped_df[affordability_features])

In [85]:
# Define weights for each feature
weights = {'IPR': 0.4, 'HPI': 0.3, 'PDI': 0.2, 'GII': 0.1}

# Calculate Affordability Score
# The Affordability Score is a weighted sum of all standardized features, where higher scores indicate better affordability
grouped_df['Affordability_Score'] = (
    weights['IPR'] * grouped_df['IPR'] +
    weights['HPI'] * grouped_df['HPI'] +
    weights['PDI'] * grouped_df['PDI'] +
    weights['GII'] * grouped_df['GII']
)

In [86]:
# Sort by Affordability Score (higher is better)
grouped_df = grouped_df.sort_values(by='Affordability_Score', ascending=False)

In [87]:
# check the top 5 most affordable areas
grouped_df.head()

,sa2_code,sa2_name,Median,extracted_price,ERP_2021,OCC_2021,population_density_persons_per_km2_2023,Gini coefficient,IPR,HPI,PDI,GII,Affordability_Score
119,209031210,Hurstbridge,57348.0,435.000000,7152.0,1.980000,237.8,0.440,1.000000,0.978410,0.097015,0.823188,0.795245
260,213051579,Manor Lakes - Quandong,60503.0,496.478261,744579.0,66.874434,166.6,0.392,0.896324,0.995038,0.138735,0.962319,0.781020
128,209041225,Whittlesea,54522.0,510.467742,667306.0,60.254972,47.1,0.434,0.739757,0.994995,0.492262,0.840580,0.776912
271,213051590,Wyndham Vale - South,53299.0,446.692308,178152.0,12.745648,2801.4,0.398,0.869859,0.996662,0.007682,0.944928,0.742971
259,213051468,Werribee - West,56515.0,497.649485,2201124.0,92.512976,220.5,0.398,0.810010,0.999286,0.104675,0.944928,0.739218


In [88]:
# check how each metric contributes to the Affordability Score
grouped_df[['sa2_code', 'sa2_name', 'IPR', 'HPI', 'PDI', 'GII', 'Affordability_Score']].head()

,sa2_code,sa2_name,IPR,HPI,PDI,GII,Affordability_Score
119,209031210,Hurstbridge,1.000000,0.978410,0.097015,0.823188,0.795245
260,213051579,Manor Lakes - Quandong,0.896324,0.995038,0.138735,0.962319,0.781020
128,209041225,Whittlesea,0.739757,0.994995,0.492262,0.840580,0.776912
271,213051590,Wyndham Vale - South,0.869859,0.996662,0.007682,0.944928,0.742971
259,213051468,Werribee - West,0.810010,0.999286,0.104675,0.944928,0.739218


## Visualization

In [89]:
# Step 1: Get the top 5 SA2 codes
top_sa2_codes = grouped_df['sa2_code'].head(5)

# Step 2: Filter the main dataset for these SA2 codes
df_top_sa2 = df[df['sa2_code'].isin(top_sa2_codes)]

# Step 3: For each SA2 code, get the first property
df_properties = df_top_sa2.groupby('sa2_code').first().reset_index()

# Step 4: Prepare the top 5 grouped DataFrame
top_grouped_df = grouped_df[['sa2_code', 'sa2_name', 'Affordability_Score']].head(10)

# Merge the dataframes on 'sa2_code'
new_df = pd.merge(top_grouped_df, df_properties[['sa2_code', 'latitude', 'longitude']], on='sa2_code')


In [90]:
# Now map the affordability score to the map
# Load the shapefile
# Import the shapefile as gdp
victoria_gdf = gpd.read_file('../data/landing/boundaries/Victoria/vic_dist_boundaries.shp')

In [91]:
# Step 1: Filter the Shapefile Data to Include Only Top 5 SA2 Areas
# Merge victoria_gdf with new_df to identify top SA2s
victoria_gdf_top5 = victoria_gdf[victoria_gdf['sa2_code'].isin(new_df['sa2_code'])]

In [92]:
# Step 2: Define a Style Function for the Top 5 Areas
def style_function(feature):
    return {
        'fillColor': 'green',  # Highlight top 5 SA2 areas in green
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7,
        'lineOpacity': 0.5,
    }

In [93]:
# Step 3: Create and Display the Map
# Get the center of the Victoria district boundaries for the map
center = victoria_gdf.geometry.centroid.y.mean(), victoria_gdf.geometry.centroid.x.mean()

# Create a Folium map centered around Victoria
m = folium.Map(location=center, zoom_start=8, tiles="OpenStreetMap")

/var/folders/gf/ml3nsztn0cb376sq34z0ll1r0000gn/T/ipykernel_31122/2842156895.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = victoria_gdf.geometry.centroid.y.mean(), victoria_gdf.geometry.centroid.x.mean()


In [94]:
# Add the top 10 district boundaries with the style function
geojson = folium.GeoJson(
    victoria_gdf_top5,
    style_function=style_function,
    tooltip=GeoJsonTooltip(fields=['sa2_name'], aliases=['District:']),
).add_to(m) 

In [95]:
# Add the top 5 SA2 areas as markers on the map
for i, row in new_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"SA2 Name: {row['sa2_name']}<br>Affordability Score: {row['Affordability_Score']:.2f}",
        icon=folium.Icon(color='green', icon='home')
    ).add_to(m)


# Display the map
m

In [323]:
# Save the results to a new CSV file
grouped_df.to_csv('sa2_affordability_scores.csv', index=False)